# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada  
### Tecnológico de Monterrey  
### Prof Luis Eduardo Falcón Morales  

### **Actividad en Equipos: sistema LLM + RAG**


## **Introducción de la problemática a resolver**

En esta actividad se implementa un chatbot basado en un modelo de lenguaje de gran tamaño (LLM) con un sistema de recuperación aumentada generativa (RAG), enfocado en facilitar el entendimiento y consulta de la normatividad aplicable en materia de competencia económica en México, con énfasis en los sectores de telecomunicaciones y radiodifusión.

El chatbot permite realizar preguntas en lenguaje natural y obtener respuestas contextualizadas con base en la Constitución, la Ley Federal de Competencia Económica, la Ley Federal de Telecomunicaciones y Radiodifusión, guías del Instituto Federal de TElecomunicaciones (IFT) y opiniones públicas relevantes.

Esta herramienta busca apoyar a analistas, abogados y personal del IFT en la interpretación y aplicación de la normatividad vigente, mejorando la eficiencia y precisión en el trabajo diario.

*Nota:* El marco normativo en esta materia se encuentra sujeto a posibles reformas. Por lo tanto, una herramienta como la que se plantea en estre traabjao deberá prever mecanismos de actualización periódica para reflejar con fidelidad el marco normativo vigente en cada momento.

## **Sistema RAG + LLM**

### **Arquitectura**

- **LLM:** GPT-4o vía API de OpenAI.
- **RAG:** Sistema de recuperación basado en embeddings de documentos legales.
- **Vector Store:** ChromaDB.
- **Embeddings:** `OpenAIEmbeddings`.
- **Carga de documentos:** Archivos PDF procesados con `pdfplumber`.

### **Pipeline**

1. Conversión de documentos PDF a texto.
2. Segmentación de texto en fragmentos manejables.
3. Creación del índice de vectores (embeddings).
4. Recuperación de fragmentos relevantes según la pregunta del usuario.
5. Generación de respuesta con GPT-4o, usando el contexto recuperado.


In [1]:
# Instalación de librerías necesarias
!pip install langchain langchain-openai langchain-community chromadb sentence-transformers pdfplumber openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 516.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.9 MB/s eta 0:00:00

In [2]:
import os
import pdfplumber

# Ruta a la carpeta con los documentos PDF
pdf_folder = "documentos_pdf"

# Lista para almacenar textos
documents = []

# Procesar cada PDF
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        with pdfplumber.open(pdf_path) as pdf:
            full_text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text + "\n"
            documents.append(full_text)

print(f"Se cargaron {len(documents)} documentos.")


Se cargaron 13 documentos.


In [8]:
import os
from google.colab import userdata

# Configurar la API Key desde los secretos de Colab
os.environ["OPENAI_API_KEY"] = userdata.get('miOpenAI_API_key')


In [9]:
# Creación del vector store

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Dividir documentos en fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_chunks = []
for doc in documents:
    docs_chunks.extend(text_splitter.split_text(doc))

print(f"Se generaron {len(docs_chunks)} fragmentos.")

# Crear embeddings
embedding_model = OpenAIEmbeddings()

# Crear vector store en memoria
vector_store = Chroma.from_texts(docs_chunks, embedding_model)

# Definir retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

print("Vector store creado y retriever listo.")


Se generaron 4893 fragmentos.
Vector store creado y retriever listo.


In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Definir el LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Crear pipeline RAG
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       retriever=retriever,
                                       return_source_documents=True)

print("Pipeline RAG + GPT-4o listo.")


Pipeline RAG + GPT-4o listo.


In [11]:
from langchain.callbacks import get_openai_callback

# Ejemplos de preguntas
preguntas = [
    "¿Qué es un agente económico con poder sustancial en México?",
    "¿Qué artículo de la LFCE regula las concentraciones?",
    "¿Qué criterios usa el IFT para evaluar barreras a la entrada?",
    "¿Cuál es el marco legal aplicable a la competencia en servicios de telecomunicaciones?",
    "¿Qué prácticas son consideradas monopólicas absolutas según la LFCE?"
]

# Hacer preguntas y medir tokens
for pregunta in preguntas:
    print(f"\nPregunta: {pregunta}")

    with get_openai_callback() as cb:
        respuesta = qa_chain({"query": pregunta})
        print(f"Respuesta: {respuesta['result']}")
        print(f"Tokens usados en esta pregunta: {cb.total_tokens}")
        print(f"Costo estimado (USD): ${cb.total_cost:.6f}")



Pregunta: ¿Qué es un agente económico con poder sustancial en México?


<ipython-input-11-3668653177>:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = qa_chain({"query": pregunta})


Respuesta: Un agente económico con poder sustancial en México es una entidad que participa en la producción, procesamiento, distribución y comercialización de bienes y servicios y que tiene la capacidad de influir significativamente en el mercado relevante. Esto se determina de acuerdo con la Ley Federal de Competencia Económica y puede incluir factores como el grado de posicionamiento de sus bienes o servicios, la falta de acceso a importaciones, la existencia de costos elevados de internación, y los diferenciales elevados en costos que enfrentan los consumidores al acudir a otros proveedores. Un agente económico con poder sustancial puede tener obligaciones específicas impuestas por el Instituto Federal de Telecomunicaciones para asegurar la competencia en el mercado.
Tokens usados en esta pregunta: 1012
Costo estimado (USD): $0.003550

Pregunta: ¿Qué artículo de la LFCE regula las concentraciones?
Respuesta: El artículo 61 de la Ley Federal de Competencia Económica (LFCE) regula las

## **Conclusiones**

- El chatbot desarrollado permite consultar de manera razonable la normatividad en materia de competencia económica en México, con énfasis en los sectores de telecomunicaciones y radiodifusión.

- La integración de un sistema RAG con un LLM como GPT-4o mejora la precisión y relevancia de las respuestas, al permitir que el modelo genere texto con base en los documentos legales específicos cargados en el corpus.

- La calidad y utilidad de las respuestas dependen directamente de la cobertura y calidad del corpus documental. En este ejercicio, la combinación de leyes, guías del IFT y opiniones públicas proporciona una base razonablemente sólida.

- Es importante señalar que el marco normativo en esta materia se encuentra sujeto a posibles reformas legislativas en el corto y mediano plazo. Por lo tanto, un despliegue operativo de una herramienta como el chatbot aquí desarrollado debería contemplar un mecanismo de actualización periódica del corpus documental, para asegurar que las respuestas se mantengan alineadas con la normatividad vigente en cada momento.

- El uso de métricas de uso de tokens y costo permite además evaluar la viabilidad económica de este tipo de soluciones en escenarios de uso intensivo.

- Este enfoque podría ser extendido a otras áreas regulatorias del IFT o de la administración pública, facilitando el acceso a marcos normativos complejos mediante interfaces de lenguaje natural.

